# Task 2: LDA Topic Model
***

We create a simple, out-of-the-box Latent Dirichlet Allocation Topic Model with visualization.

In [33]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
import numpy as np

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import CountVectorizer

In [34]:
df = pd.read_csv('cleaned_jels.csv', encoding='utf-8-sig', low_memory=False)
df.drop('0', axis=1, inplace=True)
df

,Title,Publisher,Abstract,Keywords,Jel Codes,Author1,Author2,Author3,Author4,Author5,...,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,technical efficiency of hungarian farms before...,european association of agricultural economists,hungary is one of the ten countries that have ...,"[agribusiness, production economics]",NaN,"zoltán bakucs, lajos","fertő, imre","fogarasi, józsef",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exploring the flexibility of polish family far...,european association of agricultural economists,no abstract is available for this item,"[farm management, productivity analysis]",NaN,"pieniadz, agata","renner, swetlana","petrick, martin",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,regional clusters in a function of rural devel...,european association of agricultural economists,as a theoretical concept rural development bas...,[community rural urban development],NaN,"sudarić, tihana","zmaić, krunoslav","petrač, božidar",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,the extended metropolitan area in a new member...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"zolin, m. bruna",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,infrastructural capacity of family owned holdi...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, land econo...",NaN,"Živković, dragić","dimitrijević, bojan","jelić, sreten","rajić, zoran",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,comparative research of food consumption in se...,european association of agricultural economists,no abstract is available for this item,[food consumption nutrition food safety],NaN,"Мilanović, milan r.","Đorović, milutin","stevanović, simo",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,targeting agricultural and rural development m...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"segré, andrea","rakić, renata","rokvić, gordana","vittuari, matteo",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,needs assessment analysis of small rural house...,european association of agricultural economists,no abstract is available for this item,[consumer household economics],NaN,"bogdanov, natalija","moslavac, nenad",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,designing a rural development strategy for ser...,european association of agricultural economists,no abstract is available for this item,[community rural urban development],NaN,"cochrane, nancy","reed, michael","jovanović, zlatko",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,rural development and the heritage of chayanov...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, labor and ...",NaN,"nuppenau, ernst-august",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


>We need to lemmatize all words to reduce dimensionality: jogging, jogged, jogs -> jog

In [67]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_tag(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    elif pos_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return None

class Lemmatizer_CountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(Lemmatizer_CountVectorizer, self).build_analyzer()
        return lambda doc:([lemmatizer.lemmatize(word[0], get_wordnet_tag(word[1])) for word in pos_tag(analyzer(doc)) 
                            if get_wordnet_tag(word[1]) is not None])

In [68]:
count_vectorizer = Lemmatizer_CountVectorizer(stop_words='english', lowercase=True, max_df=.5, min_df=10)
dtm_cv = count_vectorizer.fit_transform(df['Abstract'])

>We use a Latent Dirichlet Allocation Model. 

In [70]:
lda_cv = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_cv.fit(dtm_cv)

/usr/local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [71]:
pyLDAvis.sklearn.prepare(lda_cv, dtm_cv, count_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      16.310451        1       1 -0.168565 -0.001491
8      14.205317        1       2 -0.190652  0.083724
15      6.750228        1       3 -0.107954 -0.017930
16      6.288401        1       4 -0.102412  0.103072
11      6.265357        1       5 -0.144819  0.016701
9       5.944496        1       6 -0.137994 -0.101684
1       5.377901        1       7  0.307368  0.056073
4       4.966645        1       8 -0.033030 -0.245710
7       4.107362        1       9 -0.086484 -0.065639
2       3.819228        1      10 -0.063900  0.177248
17      3.663497        1      11 -0.098155  0.179562
6       3.644381        1      12 -0.057529 -0.115014
12      3.524921        1      13  0.013900 -0.121166
10      3.372285        1      14 -0.093541  0.076645
14      3.280314        1      15 -0.011906 -0.179508
13      2.154705        1      16  0.072642  0.202231
19      1.818966        1      17  0.305634  0.041812
3       1.546875        1      18  0.179590 -0.113295
0       1.488141        1      19  0.195819  0.068449
18      1.470527        1      20  0.221988 -0.044080, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
13176  Default  36334.000000         price  36334.000000  30.0000  30.0000
10834  Default  55531.000000         model  55531.000000  29.0000  29.0000
3690   Default  32391.000000       country  32391.000000  28.0000  28.0000
71     Default  23858.000000      abstract  23858.000000  27.0000  27.0000
9234   Default  23750.000000          item  23750.000000  26.0000  26.0000
10297  Default  42314.000000        market  42314.000000  25.0000  25.0000
1350   Default  25243.000000     available  25243.000000  24.0000  24.0000
6704   Default  22497.000000          firm  22497.000000  23.0000  23.0000
6467   Default  16680.000000          farm  16680.000000  22.0000  22.0000
9543   Default  18279.000000            la  18279.000000  21.0000  21.0000
17054  Default  18250.000000         trade  18250.000000  20.0000  20.0000
13841  Default  24957.000000          rate  24957.000000  19.0000  19.0000
15989  Default  16590.000000         state  16590.000000  18.0000  18.0000
1470   Default  16192.000000          bank  16192.000000  17.0000  17.0000
3995   Default  28451.000000          data  28451.000000  16.0000  16.0000
17786  Default  54863.000000           use  54863.000000  15.0000  15.0000
5981   Default  21311.000000      estimate  21311.000000  14.0000  14.0000
14721  Default  16810.000000          risk  16810.000000  13.0000  13.0000
6838   Default  13823.000000          food  13823.000000  12.0000  12.0000
6666   Default  16227.000000     financial  16227.000000  11.0000  11.0000
17872  Default  15053.000000         value  15053.000000  10.0000  10.0000
3416   Default  12020.000000      consumer  12020.000000   9.0000   9.0000
426    Default  12621.000000  agricultural  12621.000000   8.0000   8.0000
8684   Default  15288.000000   information  15288.000000   7.0000   7.0000
3654   Default  24232.000000          cost  24232.000000   6.0000   6.0000
8469   Default  16886.000000        income  16886.000000   5.0000   5.0000
15396  Default   8838.000000         shock   8838.000000   4.0000   4.0000
7601   Default  17414.000000        growth  17414.000000   3.0000   3.0000
6469   Default  10263.000000        farmer  10263.000000   2.0000   2.0000
12810  Default  37662.000000        policy  37662.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
2886   Topic20    979.956803      colombia    980.950376   4.2185  -4.6211
10945  Topic20    851.090252         moral    852.083825   4.2184  -4.7621
10313  Topic20    756.097798      marriage    757.091371   4.2182  -4.8804
2278   Topic20    693.687477           car    694.681050   4.2181  -4